## Perguntas

1) Correlação das variaveis
2) Qual estilo de jogo é mais provavel de se ter melhores vendas?
3) Qual plataforma é provavel de se ter melhores vendas?
4)  Aplicar um modelo de regressão ou classificação utilizando uma técnica de hiperparametrização automática;</br>
5)  Aplicar uma medida de avaliação do modelo.
6)  Agrupamento: gerar a visualização Scatter
7)  Classificação: gerar a matriz de confusão
8)  Regressão: gerar a visualização da linha de saída

### Consoles para analise

Wii     PS3     XBox360
WiiU    PS4     XOne

In [ ]:
# Somente rodar se necessário
# Descomente a linha a baixo para rodar!

# %pip install googletrans==4.0.0-rc1

In [1]:
#IMPORT E INSTALL AS BIBLIOTECAS IMPORTANTES
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from googletrans import Translator # Precisa do pacote do pip install

In [2]:
caminho = 'Data/vgchartz-2024.csv'
jogos = pd.read_csv(caminho)

In [4]:
jogos_tratados = jogos.drop(columns=['img','last_update'])
jogos_tratados.head(3)

,title,console,genre,publisher,developer,critic_score,total_sales,na_sales,jp_sales,pal_sales,other_sales,release_date
0,Grand Theft Auto V,PS3,Action,Rockstar Games,Rockstar North,9.4,20.32,6.37,0.99,9.85,3.12,2013-09-17
1,Grand Theft Auto V,PS4,Action,Rockstar Games,Rockstar North,9.7,19.39,6.06,0.60,9.71,3.02,2014-11-18
2,Grand Theft Auto: Vice City,PS2,Action,Rockstar Games,Rockstar North,9.6,16.15,8.41,0.47,5.49,1.78,2002-10-28


In [5]:
#TRADUZIR OS NOMES EM INGLES PARA PT-BR

def traduzir_colunas(df, language='pt'):
    translator = Translator()
    traducoes = {}
    falha_ao_traduzir = {
        'publisher': 'Editora',
        'developer': 'Desenvolvedora',
        'na_sales': 'Vendas_na',
        'jp_sales': 'Vendas_jp',
        'pal_sales': 'Vendas_pal',
        'other_sales': 'Vendas_demais_regiões',
        'critic_score': 'Pontuação_Geral'
    }
    # Traduzir todas as colunas
    for column in df.columns:
        try:
            if column.lower() in falha_ao_traduzir:
                traducoes[column] = falha_ao_traduzir[column.lower()].capitalize()  # Utilize a tradução personalizada
            else:
                translated = translator.translate(column, src='en', dest=language)
                capslock = translated.text.capitalize()  # Maiuscula na primeira letra de cada palavra
                traducoes[column] = capslock
        except Exception as e:
            print(f"Erro ao traduzir a coluna '{column}': {e}")
            traducoes[column] = column  # Manter o nome original em caso de erro
    df.columns = [traducoes.get(col, col) for col in df.columns]

In [6]:
#EXECUTA A FUNÇÃO DE TRADUÇÃO
traduzir_colunas(jogos_tratados)
jogos_tratados.head(3)

,Título,Console,Gênero,Editora,Desenvolvedora,Pontuação_geral,Vendas totais,Vendas_na,Vendas_jp,Vendas_pal,Vendas_demais_regiões,Data de lançamento
0,Grand Theft Auto V,PS3,Action,Rockstar Games,Rockstar North,9.4,20.32,6.37,0.99,9.85,3.12,2013-09-17
1,Grand Theft Auto V,PS4,Action,Rockstar Games,Rockstar North,9.7,19.39,6.06,0.60,9.71,3.02,2014-11-18
2,Grand Theft Auto: Vice City,PS2,Action,Rockstar Games,Rockstar North,9.6,16.15,8.41,0.47,5.49,1.78,2002-10-28


In [7]:
# Contagem dos lavores zerados no df tratado
jogos_tratados.isna().sum()

Título                       0
Console                      0
Gênero                       0
Editora                      0
Desenvolvedora              17
Pontuação_geral          57338
Vendas totais            45094
Vendas_na                51379
Vendas_jp                57290
Vendas_pal               51192
Vendas_demais_regiões    48888
Data de lançamento        7051
dtype: int64

In [8]:
# Separação de valores ausentes para análise
valores_ausentes = jogos_tratados[jogos_tratados['Desenvolvedora'].isna()]
valores_ausentes.head(18)

,Título,Console,Gênero,Editora,Desenvolvedora,Pontuação_geral,Vendas totais,Vendas_na,Vendas_jp,Vendas_pal,Vendas_demais_regiões,Data de lançamento
8486,Gourmet Chef: Cook Your Way to Fame,DS,Misc,Ubisoft,NaN,NaN,0.14,0.13,NaN,NaN,0.01,2008-06-24
10832,Wordmaster,DS,Strategy,"Destination Software, Inc",NaN,NaN,0.09,0.09,NaN,0.0,0.01,2008-08-12
18185,SAS: Secure Tomorrow,PC,Shooter,City Interactive,NaN,NaN,0.00,NaN,NaN,0.0,0.00,2008-10-16
18455,My Baby and Me,Wii,Misc,Unknown,NaN,NaN,0.00,NaN,NaN,0.0,NaN,NaN
19925,Bounty Hunter,PC,Action,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999-11-30
25045,Abandoner,PC,Adventure,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2004-09-17
27407,The Enchanted Unicorn,PC,Adventure,Take-Two Interactive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008-09-16
31230,istanbul Beyleri,PC,Fighting,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-12-17
37057,Purrfect Pet Shop,PC,Misc,eGames,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008-06-13
40753,Duludubi Star,PC,Platform,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Separação dos consoles que pretendemos trabalhar
consoles = ('PS3','PS4','X360','XOne','Wii','WiiU')
consoles_tratados = jogos_tratados[(jogos_tratados['Console'].isin(consoles))]
consoles_tratados.isna().sum()

Título                      0
Console                     0
Gênero                      0
Editora                     0
Desenvolvedora              2
Pontuação_geral          8909
Vendas totais            5124
Vendas_na                6218
Vendas_jp                8737
Vendas_pal               6785
Vendas_demais_regiões    5765
Data de lançamento       1916
dtype: int64

In [14]:
# Visualização das infos dos consoles
consoles_tratados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10711 entries, 0 to 64015
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Título                 10711 non-null  object 
 1   Console                10711 non-null  object 
 2   Gênero                 10711 non-null  object 
 3   Editora                10711 non-null  object 
 4   Desenvolvedora         10709 non-null  object 
 5   Pontuação_geral        1802 non-null   float64
 6   Vendas totais          5587 non-null   float64
 7   Vendas_na              4493 non-null   float64
 8   Vendas_jp              1974 non-null   float64
 9   Vendas_pal             3926 non-null   float64
 10  Vendas_demais_regiões  4946 non-null   float64
 11  Data de lançamento     8795 non-null   object 
dtypes: float64(6), object(6)
memory usage: 1.1+ MB


In [17]:
# Visualização das infos dos consoles
consoles_tratados.groupby(['Console'])['Console'].count()

Console
PS3     1905
PS4     2878
Wii     1676
WiiU     554
X360    1735
XOne    1963
Name: Console, dtype: int64

In [20]:
# Visualização das infos dos consoles
jogos_NA = consoles_tratados.groupby('Console').apply(lambda x: x.isna().sum())
jogos_NA.head(6)

,Título,Console,Gênero,Editora,Desenvolvedora,Pontuação_geral,Vendas totais,Vendas_na,Vendas_jp,Vendas_pal,Vendas_demais_regiões,Data de lançamento
Console,,,,,,,,,,,,
PS3,0,0,0,0,0,1390,557,938,1195,923,813,166
PS4,0,0,0,0,0,2730,1970,2191,2398,2396,2120,776
Wii,0,0,0,0,2,1329,319,561,1421,741,440,83
WiiU,0,0,0,0,0,473,406,442,501,433,421,135
X360,0,0,0,0,0,1096,433,615,1288,665,531,139
XOne,0,0,0,0,0,1891,1439,1471,1934,1627,1440,617
